In [1]:
    try:
        from google.colab import drive

        print('Colab environment detected. Mounting drive...')
        drive.mount('/content/drive')

        print('Mounted. Switching to directory... ', end = '')
        %cd /content/drive/'My Drive'/CILroadseg
        print('done.')
    except:
        print('Colab environment not found. Working on ordinary directory.')

Colab environment detected. Mounting drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted. Switching to directory... /content/drive/My Drive/CILroadseg
done.


In [2]:
import numpy as np
import sys
import os
import matplotlib.image as mpimg

from util.submit import *
from util.helpers import *
from util.visualize import *
from util.notebooks import *
from util.cross_validation import cross_validate

## Loading Training Data

In [3]:
train_dir = "training/images/"
gt_dir = "training/groundtruth/"
test_dir = "test/images/"

X, Y, X_test = nb_load_data(train_dir, gt_dir, test_dir)

Loading training input...
Progress: done (100 images).
Loading training groundtruth...
Progress: done (100 images).
Loading test input...
Progress: done (94 images).

       Training data shape: (100, 400, 400, 3)
Training groundtruth shape: (100, 400, 400)
           Test data shape: (94, 608, 608, 3)


## Decomposer + ResNet/CNN

In [4]:
import tensorflow as tf
from tensorflow import keras

from discretize import *
from rotate_vote import *
from unet import *

Using TensorFlow backend.


In [5]:
core_model = UNetModel()

model = core_model
model_dsc = Discretizer(core_model)

In [ ]:
model_id = "unet1"

choice = 'tr'

np.random.seed(1) #fix randomness
if choice == 'cv':
  cross_validate(model, 4, X, Y)

elif choice == 'tr':
  model.initialize()
  model.train(Y, X) 

  nb_save_model(model, "saves/final/"+ model_id +".h5") 

else:
  model.initialize()
  model.load("saves/final/"+ model_id +".h5")

y
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400, 400, 3)  0                                            
__________________________________________________________________________________________________
conv_block_0 (Sequential)       (None, 400, 400, 64) 38720       input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 200, 200, 64) 0           conv_block_0[1][0]               
__________________________________________________________________________________________________
conv_block_1 (Sequential)       (None, 200, 200, 128 221440      max_pooling2d_1[0][0]            
__________________________________________________________________________________________

## Submission Generation

In [ ]:
test_masks_dir = "test/pred/"
test_dir = "test/images/"
 
nb_predict_masks(model_dsc, test_dir, test_masks_dir + "unet/")

In [ ]:
image_paths = [test_masks_dir + "unet/" + file for file in os.listdir(test_masks_dir + "unet/")]
masks_to_submission("test/"+ model_id +".csv", image_paths)

In [ ]:
Y_pred = model.classify(X_test[0:10])
Y_pred_dsc = model_dsc.classify(X_test[0:10])

view_image_array(X_test[0:10], Y_pred, Y_pred_dsc)